In [1]:
from rdflib.namespace import RDF
from rdflib.graph import DATASET_DEFAULT_GRAPH_ID
from rdflib import Graph, Namespace, Literal, URIRef, ConjunctiveGraph
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore
from rdflib.plugins.sparql import prepareQuery
import requests

In [64]:
ns = Namespace("http://example.org/")
dataset = "test"
dataset_type = "tdb2"  # tdb, tdb1, or tdb2
base_url = "http://toolsfrek1.oeko.local/fuseki"
endpoint = f"{base_url}/{dataset}"
admin_list = f"{base_url}/$/datasets"
admin_details = f"{admin_list}/{dataset}"
graph_id = ns.mygraph  # must be IRI

In [94]:
# delete old dataset
requests.delete(admin_details)
# create dataset (https://jena.apache.org/documentation/fuseki2/fuseki-server-protocol.html#datasets-and-services)
requests.post(
    admin_list, data={"dbType": dataset_type, "dbName": dataset}
).raise_for_status()

In [95]:
store = SPARQLUpdateStore(endpoint)
store.open((f"{endpoint}/query", f"{endpoint}/update"))
cgraph = ConjunctiveGraph(store=store)
dgraph = Graph(store=store, identifier=DATASET_DEFAULT_GRAPH_ID)
graph = Graph(store=store, identifier=graph_id)

In [96]:
query1 = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o
}
"""

query2 = (
    """
SELECT ?s ?p ?o
FROM <%s>
WHERE {
  ?s ?p ?o
}
"""
    % graph_id
)

In [97]:
# add node
node1 = (ns.me, ns.verb, Literal("mytype"))
node2 = (ns.me, ns.verb, Literal("mytype2"))
node3 = (ns.me, ns.verb, URIRef("http://example.org/a"))

graph.add(node2)
graph.add(node1)
dgraph.add(node1)
dgraph.add(node3)

<Graph identifier=urn:x-rdflib:default (<class 'rdflib.graph.Graph'>)>

In [103]:
res = list(cgraph.triples((None, None, None)))
print(len(res))
print(res)

2
[(rdflib.term.URIRef('http://example.org/me'), rdflib.term.URIRef('http://example.org/verb'), rdflib.term.Literal('mytype')), (rdflib.term.URIRef('http://example.org/me'), rdflib.term.URIRef('http://example.org/verb'), rdflib.term.URIRef('http://example.org/a'))]


In [104]:
res = list(dgraph.triples((None, None, None)))
print(len(res))
print(res)

2
[(rdflib.term.URIRef('http://example.org/me'), rdflib.term.URIRef('http://example.org/verb'), rdflib.term.Literal('mytype')), (rdflib.term.URIRef('http://example.org/me'), rdflib.term.URIRef('http://example.org/verb'), rdflib.term.URIRef('http://example.org/a'))]


In [105]:
res = list(graph.triples((None, None, None)))
print(len(res))
print(res)

2
[(rdflib.term.URIRef('http://example.org/me'), rdflib.term.URIRef('http://example.org/verb'), rdflib.term.Literal('mytype2')), (rdflib.term.URIRef('http://example.org/me'), rdflib.term.URIRef('http://example.org/verb'), rdflib.term.Literal('mytype'))]


In [93]:
res = list(graph.query(query1))
print(len(res))
print(res)

2
[(rdflib.term.URIRef('http://example.org/me'), rdflib.term.URIRef('http://example.org/verb'), rdflib.term.URIRef('http://example.org/a')), (rdflib.term.URIRef('http://example.org/me'), rdflib.term.URIRef('http://example.org/verb'), rdflib.term.Literal('mytype'))]
